In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Show code')

In [3]:
# Para eliminar warnings
import warnings
warnings.filterwarnings("ignore")

# Trabajo Práctico 3

## Parte 1

### Ejercicio 2

In [4]:
# Importamos los paquetes que vamos a utilizar

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from scipy.special import expit

import sklearn as sks
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib.pyplot as plt

In [16]:
eph_hogar = pd.read_excel('usu_hogar_T121.xlsx')
eph_individual = pd.read_excel('usu_individual_T121.xlsx')

In [17]:
# A partir del TP2, la única condición que usaremos será que la región sea Gran Buenos Aires

eph_individual = eph_individual[eph_individual['REGION'] == 1]
eph_hogar = eph_hogar[eph_hogar['REGION'] == 1]

### Ejercicio 3

In [18]:
# Unimos la encuesta individual con la del hogar

eph = pd.merge(eph_hogar, eph_individual, how = 'left', on = ['CODUSU', 'NRO_HOGAR'])

### Ejercicio 4

In [19]:
# Esta función (ya creada) borra las columnas que estén completamente llenas de NaN
# Con axis, están los valores 0 y 1: el default (0) elimina si el valor es nulo, pero seteando en (1) se borran missing values
# Se podrían borrar las columnas con al menos un NaN, indicando how = "any" y sino las que sean todas NaN con "all"

eph_sinnan = eph.dropna(axis=1, how="all")

print("El número de columnas del df eliminando las columnas completamente de NaN es", len(eph_sinnan.columns))
print("El número de columnas de la base original es", len(eph.columns))

El número de columnas del df eliminando las columnas completamente de NaN es 248
El número de columnas de la base original es 263


### Ejercicio 5

In [21]:
# Separamos variables categóricas de no categóricas

eph_categoricas= eph.drop(['CODUSU', 'CH05','MAS_500_x','ANO4_x','MAS_500_y','IDECOCUR', 'IDECIFR_y', 'PP09C_ESP', 'PP09A_ESP', 
'IDECCFR_y', 'PDECOCUR', 'PDECCFR_y', 'IDECINDR', 'II8_ESP', 'PDECINDR', 'II7_ESP', 'IV7_ESP', 'IV3_ESP', 'IV1_ESP', 'PDECIFR_y', 
'PDECIFR_x', 'ADECIFR_x','IPCF_x', 'DECCFR_x', 'IDECCFR_x', 'RDECCFR_x', 'GDECCFR_x', 'PDECCFR_x', 'ADECCFR_x', 'PONDIH_x',
'REALIZADA','REGION_y', 'AGLOMERADO_y', 'PONDERA_y','ANO4_y','TRIMESTRE_y','ITF_y','DECIFR_y','RDECIFR_y','GDECIFR_y','ADECIFR_y',
'IPCF_y','DECCFR_y','RDECCFR_y','GDECCFR_y','ADECCFR_y','PONDIH_y','TRIMESTRE_x', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'REGION_x', 
'AGLOMERADO_x', 'PONDERA_x','PP04D_COD','PP04B_COD','CH06', 'IMPUTA', 'CH16_COD','PP11D_COD', 'PP11C', 'PP11C99',    
'PP11B2_DIA',    
'PP11B2_MES',    
'PP11B2_ANO',    
'PP11T',    
'PP11B_COD',    
'PP11A',    
'PP10E',    
'PP10D',    
'PP10C',    
'PP10A',    
'PP11B1',    
'PP11G_ANO',    
'PP11N',    
'PP11G_MES',    
'PP11S',    
'PP11R',    
'PP11Q',    
'PP11P',    
'PP11O',    
'PP11M',    
'PP11L1','PP11L', 'PP11G_DIA', 'CH15_COD','PP03D', 'PP3E_TOT', 'PP3F_TOT', 'PP04B2', 'PP06C', 'PP06D', 'PP08D1', 'PP08D4', 
'PP08F1', 'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'P21', 'DECOCUR','RDECOCUR', 'GDECOCUR', 'ADECOCUR', 'PONDIIO', 'TOT_P12', 
'P47T', 'DECINDR', 'RDECINDR', 'GDECINDR', 'ADECINDR', 'PONDII', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M',
'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI', 'ITF_x', 'DECIFR_x', 'IDECIFR_x', 'RDECIFR_x', 'GDECIFR_x'], axis=1)

In [22]:
# Todas las categóricas comienzan con el valor 1 en adelante (hasta 2, 3, 6...)
# Por lo tanto, borramos los missing values y los reemplazamos por cero en este subconjunto de variables

eph_categoricas=eph_categoricas.fillna(0)

In [23]:
# Para trabajar mejor con las variables, transformamos las categóricas en dummies según cada caso

eph_dummies_categoricas = pd.get_dummies(eph_categoricas, 
                            columns=['PP02C1', 'PP02C2', 'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6',
                            'PP02C7', 'PP02C8', 'PP02E', 'PP02H', 'PP02I', 'PP03C', 'PP03G', 'PP03H', 'PP03I', 'PP03J', 'INTENSI', 'PP04A', 
                            'PP04B1', 'PP04B3_MES', 'PP04B3_ANO', 'PP04B3_DIA', 'PP04C', 'PP04C99', 'PP04G', 'PP05B2_MES', 'PP05B2_ANO',
                            'PP05B2_DIA', 'PP05C_1', 'PP05C_2', 'PP05C_3', 'PP05E', 'PP05F', 'PP05H', 'PP06A', 'PP06E', 'PP06H', 'PP07A', 
                            'PP07C', 'PP07D', 'PP07E', 'PP07F1', 'PP07F2', 'PP07F3', 'PP07F4', 'PP07F5', 'PP07G1', 'PP07G2', 'PP07G3',
                            'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K', 'PP09A', 'PP09B', 'PP09C', 'IV1', 'IV2', 'IV3', 'IV4', 
                            'IV5', 'IV6', 'IV7', 'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2', 'II3', 'II3_1', 
                            'II4_1', 'II4_2', 'II4_3', 'II5', 'II5_1', 'II6', 'II6_1', 'II7', 'II8', 'II9', 'V1', 'V2', 'V21', 'V22', 'V3',
                            'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A', 
                            'V19_B', 'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10', 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4',
                            'CH03', 'CH07', 'CH08', 'CH09', 'CH10', 'CH11', 'CH12','CH13', 'CH14', 'CH15', 
                            'CH16', 'NIVEL_ED', 'ESTADO', 'CAT_OCUP', 'CAT_INAC'])

In [24]:
eph_dummies_categoricas['CH04'] = eph['CH04']

In [25]:
eph_no_categoricas=eph.loc[:,['CH06','PP03D', 'PP3E_TOT', 'PP3F_TOT', 'PP04B2', 'PP06C', 'PP06D', 'PP08D1', 'PP08D4', 'PP08F1', 
                                    'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'P21', 'DECOCUR','RDECOCUR', 'GDECOCUR', 'ADECOCUR', 
                                    'PONDIIO', 'TOT_P12', 'P47T', 'DECINDR', 'RDECINDR', 'GDECINDR', 'ADECINDR', 'PONDII', 'V2_M',
                                    'V3_M', 'PDECOCUR', 'PDECINDR', 'IDECINDR', 'IDECOCUR',
                                    'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI',
                                    'ITF_x', 'DECIFR_x', 'IDECIFR_x', 'RDECIFR_x', 'GDECIFR_x', 'PDECIFR_x', 'ADECIFR_x','IPCF_x', 
                                    'DECCFR_x', 'IDECCFR_x', 'RDECCFR_x', 'GDECCFR_x', 'PDECCFR_x', 'ADECCFR_x', 'PONDIH_x']]

In [26]:
eph_no_categoricas['CODUSU'] = eph['CODUSU']
eph_no_categoricas['NRO_HOGAR'] = eph['NRO_HOGAR']

In [27]:
# Por su parte, a los missing values de las variables no categóricas los reemplazaremos por la media

miss_mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [28]:
# Por su parte, las variables categóricas son en su mayoría variables ingreso, nos quedaremos entre los cuantiles 0.99 y 0
# y mantenemos a los de menores ingresos

for column in eph_dummies_categoricas:
    # Definimos los límites y luego lo aplicamos a los datos
    limite_inferior_categ = eph_dummies_categoricas[column].quantile(0)
    limite_superior_categ = eph_dummies_categoricas[column].quantile(0.99)
    
    eph_dummies_categoricas_final = eph_dummies_categoricas[(eph_dummies_categoricas[column] <= limite_superior_categ) & (eph_dummies_categoricas[column] >= limite_inferior_categ)]

In [29]:
# Para borrar aquellas variables cuyos valores son en su mayoría NaN (elegimos el valor 0.8)

limitPer = len(eph) * .80
eph = eph.dropna(thresh=limitPer, axis=1)

print("El número de columnas de la base ahora es", len(eph.columns))

El número de columnas de la base ahora es 156


In [30]:
eph_final = pd.concat([eph_no_categoricas, eph_dummies_categoricas_final], axis=1)

### Ejercicio 6

In [31]:
# Mostramos la estadística descriptiva de 5 variables que podrían ser relevantes para predecir la pobreza.

eph[['NIVEL_ED', 'ESTADO', 'IV8', 'IV11', 'IV12_3']].describe()

# Otras variables importantes también podrían ser: V6, V19_B

,NIVEL_ED,ESTADO,IV8,IV11,IV12_3
count,4082.000000,4082.000000,4082.000000,4082.000000,4082.000000
mean,3.827291,2.184468,1.001715,1.389760,1.990691
std,1.756232,1.115713,0.041380,0.627885,0.096046
min,1.000000,0.000000,1.000000,0.000000,1.000000
25%,3.000000,1.000000,1.000000,1.000000,2.000000
50%,4.000000,3.000000,1.000000,1.000000,2.000000
75%,5.000000,3.000000,1.000000,2.000000,2.000000
max,7.000000,4.000000,2.000000,3.000000,2.000000


### Ejercicio 7

In [33]:
eph_equiv = pd.read_excel('tabla_adulto_equiv.xlsx', header=3, skipfooter=12)
eph_equiv.columns=['Edad', 'Mujeres', 'Varones', 'Lista']#, 'a', 'a', 'a', 'a', 'a', 'a', 'a']
eph_equiv=eph_equiv.drop(0)
#eph_equiv=eph_equiv.drop('a', axis=1)
eph_equiv=eph_equiv.reset_index()
eph_equiv=eph_equiv.drop('index',axis=1)
eph_equiv

edades=[*range(18),[*range(18,30)],[*range(30,46)],[*range(46,61)],[*range(61,76)], [*range(75,120)]]
#edades

eph_equiv['Lista']=edades
eph_equiv

# Sexo. 01 = Varón. 02 = Mujer


#for row in df_equiv.iterrows():
    # row[1][1] es el valor de mujer
    # row[1][2] es el valor de hombre
    # row[1][3] es el valor de las edades (enteros o lista)

,Edad,Mujeres,Varones,Lista
0,Menor de 1 año,0.35,0.35,0
1,1año,0.37,0.37,1
2,2 años,0.46,0.46,2
3,3 años,0.51,0.51,3
4,4 años,0.55,0.55,4
5,5 años,0.60,0.60,5
6,6 años,0.64,0.64,6
7,7 años,0.66,0.66,7
8,8 años,0.68,0.68,8
9,9 años,0.69,0.69,9


In [35]:
def adulto_equiv(sexo, edad):
    """
    Encuentra los valores de necesidades energéticas para el sexo y rango etario.
    ### Parámetros
    - sexo (int): 1 corresponde a varón y 2 a mujer.
    - edad (int): edad.
    ### Devuelve
    El float correspondiente a la necesidad energética.
    ### Uso
    val = adulto_equiv(s, e)
    """
    ix=eph_equiv.index[eph_equiv['Lista']==edades]
    #ix=ix.values[0]
    if len(ix)==0:
        for i in eph_equiv.iterrows():
            try:
                if edad in i[1][3]:
                    ix = i[0]
            except:
                pass
    else:
        ix=ix.values[0]
    
    if sexo==1:
        return eph_equiv.iloc[ix][2]
    elif sexo==2:
        return eph_equiv.iloc[ix][1]

In [36]:
# indices de las columnas
pd.Index.get_loc(eph_final.columns,key='CH04')
pd.Index.get_loc(eph_final.columns,key='CH06')

eph_final['adulto_equiv']=0

In [37]:
#df['adulto_equiv']= adulto_equiv(df['CH04'], df['CH06'])
#ad_equiv_list=[]
#for row in eph_final.iterrows():
    #ad_equiv_list.append(adulto_equiv(
        #row[1][pd.Index.get_loc(eph_final.columns,key='CH04')] and
        #row[1][pd.Index.get_loc(eph_final.columns,key='CH06')]))

ad_equiv_list=[]
for row in eph_final.iterrows():
    ad_equiv_list.append(adulto_equiv(
        row[1][pd.Index.get_loc(eph_final.columns,key='CH04')],
        row[1][pd.Index.get_loc(eph_final.columns,key='CH06')]))

eph_final['adulto_equiv'] = ad_equiv_list

In [38]:
# adultos_por_hogar=df_agrup_por_hogar['adulto_equiv'].agg(np.sum)
# adultos_por_hogar.plot(kind='bar')
# plt.show()
eph_final['ad_equiv_hogar'] = eph_final['adulto_equiv'].groupby(eph_final['CODUSU']).transform(np.sum)
eph_final[['CODUSU', 'adulto_equiv', 'ad_equiv_hogar']]

,CODUSU,adulto_equiv,ad_equiv_hogar
0,TQRMNOQXUHJMMUCDEIJAH00693031,0.35,1.40
1,TQRMNOQXUHJMMUCDEIJAH00693031,0.35,1.40
2,TQRMNOQXUHJMMUCDEIJAH00693031,0.35,1.40
3,TQRMNOQXUHJMMUCDEIJAH00693031,0.35,1.40
4,TQRMNOSRPHLLNRCDEIJAH00651171,0.35,1.40
...,...,...,...
4077,TQRMNORRWHJMOPCDEIIAD00690119,0.35,0.35
4078,TQRMNOQWRHLOMPCDEIJAH00650932,0.35,1.05
4079,TQRMNOQWRHLOMPCDEIJAH00650932,0.35,1.05
4080,TQRMNOQWRHLOMPCDEIJAH00650932,0.35,1.05


### Ejercicio 8

In [39]:
# Base de datos con observaciones donde respondieron la pregunta sobre su ITF
respondieron = (eph_final.loc[eph_final['ITF_x']!=0]).copy()

# Base de datos con observaciones donde no respondieron la pregunta sobre su ITF
norespondieron = (eph_final.loc[eph_final['ITF_x']==0]).copy()

In [40]:
respondieron['ingreso_necesario'] = respondieron['ad_equiv_hogar'] * 18914

### Ejercicio 9

In [41]:
respondieron['pobre'] = np.nan

In [42]:
respondieron.loc[respondieron['ITF_x']  > respondieron['ingreso_necesario'], 'pobre'] = 0
respondieron.loc[respondieron['ITF_x']  < respondieron['ingreso_necesario'], 'pobre'] = 1

n_pobres = len(respondieron[respondieron['pobre'] == 1])
print('La cantidad de pobres es:', n_pobres)

La cantidad de pobres es: 274


### Ejercicio 10

In [43]:
# Para utilizar una sola variable por hogar sumando el ponderador

base_individuo = respondieron.groupby(['CODUSU','NRO_HOGAR']).agg({"PONDIH_x" : "sum", "pobre" : "mean"})
base_individuo.groupby("pobre").PONDIH_x.sum()

pobre
0.0    13550653
1.0     1801369
Name: PONDIH_x, dtype: int64

In [44]:
tasa_hogares_pobreza = 1801369/(13550653+1801369)
print("La tasa de hogares bajo la línea de pobreza es del", tasa_hogares_pobreza, "%.")

La tasa de hogares bajo la línea de pobreza es del 0.1173375728617377 %.


## Parte 2

### Ejercicio 1

In [45]:
def evalua_metodo(X_train, X_test, y_train, y_test, model, verbose=True):

    '''
    DEVUELVE EL ACCURACY DEL METODO ELEGIDO, EL FPR Y EL FNR
    params: cada modelo junto con su muestra de entrenamiento y de testeo.
    return: para cada modelo, se imprime la matriz de confusión, la curva ROC y los valores AUC y accuracy score.
    '''
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_true=y_test, y_score=y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)

    
    if verbose:
        print(confusion_matrix(y_test, y_pred))
        plt.plot(fpr, tpr)
    else:
        pass
    
    return auc, fpr, tpr

### Ejercicio 2

In [47]:
def cross_validation(modelo, X, y, k=5):
    """ 
    Realiza la validación cruzada para cada una de las k particiones.
        Inputs:
                - Cada modelo
                - Un valor de K
                - Un dataset: X e y.
        Output:
    """
    kf=KFold(n_splits=k, shuffle=True, random_state=1)
    kf.get_n_splits(X)
    KFold(n_splits=k, shuffle=False)
    
    X=X.to_numpy()
    y=y.to_numpy()
    
    auc_list=[]
    # este loop  de la documentación de KFold
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(X_train.shape)
        #print(X_test.shape)
        #print(y_train.shape)
        #print(y_test.shape)
        auc, _, _ = evalua_metodo(X_train, X_test, y_train, y_test, modelo, False)
        auc_list.append(auc)
    
    return auc_list

### Ejercicio 3

In [49]:
def evalua_config(X, y, model_string, dict_config, accs):
    """
    Inputs:
    X = variables dependientes
    y = variables a predecir
    
    model_string = string que indica el tipo de modelos.    
    Acepta: "KNN", "LDA", "Logreg", 'DTree', "SVM", "Bag", "RF", "Boost"
    
    dict_config =   diccionario, requiere tener como keys el string del modelo y la configuracion
                    como  
                    "Lin": Intercepto o no
                    "KNN" : Cantidad de vecinos cercanos (sugerido 1 a 10)
                    "LDA" : FALTA
                    "Logreg" : FALTA 
                    'DTree' : FALTA 
                    "SVM" : FALTA 
                    "Bag" : base_estimator, default=None
                            n_estimators, default=10 (sugerido entre 10^0 a 10^2)
                    "RF" : n_estimators, default=100 (sugerido entre 10^1 a 10^5) 
                    "Boost" : learning_rate: default=0.1 (sugerido entre 0 y 1)
                    
    accs = diccionario donde se guardan los resultados
    
    """
    
    params=dict_config.values()
    for name, parameters in dict_config.items():
        modelnames={"Lin": LinearRegression(*parameters),
                    "KNN" : KNeighborsClassifier(*parameters), 
                    "LDA" : LinearDiscriminantAnalysis(*parameters),
                    "Logreg" : LogisticRegression(*parameters), 
                    "DTree" : DecisionTreeClassifier(*parameters), 
                    "SVM" : LinearSVC(*parameters), 
                    "Bag" : BaggingClassifier(*parameters), 
                    "RF" : RandomForestClassifier(*parameters), 
                    "Boost" : GradientBoostingClassifier(*parameters)}
        model = modelnames[model_string]
        # necesitamos parametrizar al modelo antes de inicializarlo
        accs[name] = cross_validation(model, X, y, k=5)

### Ejercicio 4

In [50]:
def evalua_multiples_metodos(X, y, model_list):
	#import warnings
	#warnings.filterwarnings("ignore", category=FutureWarning)
	# model_list es una lista con diccionarios dentro
	accs={}
	for models in model_list:
		for name, model in models.items():
			evalua_config(X, y, name, model, accs)
			# le paso el diccionario para que evalua_config me lo rellene
            accsdf=(pd.DataFrame(accs)).mean()
	return accs, accsdf

## Parte 3

### Ejercicio 1

In [51]:
columns_to_drop = ['PP06C', 'IDECIFR_x','PP06D', 'PP08D1', 'PP08D4', 'PP08F1', 'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'P21', 'DECOCUR', 'IDECOCUR', 'RDECOCUR', 'GDECOCUR', 'PDECOCUR', 'ADECOCUR', 'PONDIIO','TOT_P12','P47T', 'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 'ADECINDR','V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI','ITF', 'DECIFR', 'IDECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR','IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR','adulto_equiv', 'ad_equiv_hogar', 'ingreso_necesario']    

In [52]:
for i in columns_to_drop:
    try:
        norespondieron = norespondieron.drop(i,axis=1)
    except:
        pass

In [53]:
for i in columns_to_drop:
    try:
        respondieron = respondieron.drop(i,axis=1)
    except:
        pass

In [54]:
x = respondieron.drop('pobre', axis=1)
x = x.select_dtypes(exclude=['object'])
x = x.fillna(x.mean())
x=x.dropna(axis=1)

y = respondieron.pobre

x=x.reset_index()
x=x.drop('index', axis=1)
y=y.reset_index()
y=y.drop('index', axis=1)

### Ejercicio 2

In [55]:
knn_dict={"KNN3":[3], 
          "KNN4":[4]}

logreg_dict={"Logreg3":["none"], "Logreg4":["l2"]}

dtree_dict={"DTree_gini":["gini"], "DTree_entropy":["entropy"]}

lda_dict= {"LDA_LSQR":['lsqr'],
           "LDA_eigen":['svd']}

svm_dict= {"SVM_1": ["l2"],
          "SVM_2": ["l2"]}

bag_dict= {"Bag_DeTree": [None],#, 10 ], 
           "Bag_SVC":[SVC()]}#, 100]}

rf_dict={"RF_100":[100],#,"gini"], 
         "RF_1000":[1000]}#,"gini"]}

boost_dict= {"Boost_dev":['deviance'],#,0.1], 
             "Boost_expo":['exponential']}#,0.05]}

Lin_dict= {"Lin_yes":[True],#,0.1], 
           "Lin_no":[False]}#,0.05]}

accs={}

In [56]:
model_list=[{"KNN":knn_dict},
            {"Lin":Lin_dict},
            {"LDA":lda_dict},
            {"Logreg" : logreg_dict},
            {"DTree": dtree_dict},
            {"SVM" : svm_dict},
            {"Bag" : bag_dict},
            {"RF" : rf_dict},
            {"Boost" : boost_dict}
           ]
accs=evalua_multiples_metodos(x, y, model_list)
accs

{'KNN3': [0.9705919185172818,
  1.0,
  0.9989082969432315,
  0.9814814814814814,
  0.9988764044943821],
 'KNN4': [0.9716981132075472,
  0.9508196721311475,
  0.9989082969432315,
  0.9814814814814814,
  0.9819272519520093],
 'Lin_yes': [0.9638086491901819,
  0.9829419583517944,
  0.978909226052216,
  0.9579835390946503,
  0.978137497619501],
 'Lin_no': [0.9633912172315913,
  0.982868114015655,
  0.978909226052216,
  0.9579835390946503,
  0.978137497619501],
 'LDA_LSQR': [0.8651903489731174,
  0.8680401713188598,
  0.9347997770138438,
  0.8799999999999999,
  0.8616454008760235],
 'LDA_eigen': [0.8662965436633828,
  0.869166297444986,
  0.9337080739570752,
  0.8777777777777777,
  0.8785945534183965],
 'Logreg3': [0.9484054099181833,
  0.9217250036922168,
  0.962626591099136,
  0.9896296296296296,
  0.8723671681584461],
 'Logreg4': [0.976707296710636,
  0.9135282823807414,
  0.9732648889714764,
  0.97,
  0.9819272519520093],
 'DTree_gini': [1.0, 1.0, 1.0, 1.0, 1.0],
 'DTree_entropy': [1.0,

In [73]:

df

KNN3             0.989972
KNN4             0.976967
Lin_yes          0.972356
Lin_no           0.972258
LDA_LSQR         0.881935
LDA_eigen        0.885109
Logreg3          0.938951
Logreg4          0.963086
DTree_gini       1.000000
DTree_entropy    1.000000
SVM_1            0.834965
SVM_2            0.804971
Bag_DeTree       1.000000
Bag_SVC          0.883568
RF_100           1.000000
RF_1000          1.000000
Boost_dev        1.000000
Boost_expo       1.000000
dtype: float64

## Inciso 9

In [58]:
rf=RandomForestClassifier().fit(x, y)
y_pred = np.sum(rf.predict(x))/x.shape[1]
y_pred

0.4356120826709062

## New heading

In [59]:
norespondieron

,CH06,PP03D,PP3E_TOT,PP3F_TOT,PP04B2,PONDII,ITF_x,DECIFR_x,RDECIFR_x,GDECIFR_x,...,CAT_OCUP_4,CAT_OCUP_9,CAT_INAC_0,CAT_INAC_1,CAT_INAC_2,CAT_INAC_3,CAT_INAC_4,CAT_INAC_5,CAT_INAC_6,CAT_INAC_7
18,41,0.0,40.0,0.0,0.0,0,0.0,12.0,12.0,12.0,...,0,0,1,0,0,0,0,0,0,0
19,42,0.0,40.0,0.0,0.0,0,0.0,12.0,12.0,12.0,...,1,0,1,0,0,0,0,0,0,0
20,24,NaN,NaN,NaN,NaN,4300,0.0,12.0,12.0,12.0,...,0,0,0,0,0,1,0,0,0,0
21,19,NaN,NaN,NaN,NaN,4300,0.0,12.0,12.0,12.0,...,0,0,0,0,0,1,0,0,0,0
22,17,NaN,NaN,NaN,NaN,4300,0.0,12.0,12.0,12.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,57,0.0,0.0,0.0,0.0,2703,0.0,12.0,12.0,12.0,...,0,0,1,0,0,0,0,0,0,0
4065,53,0.0,12.0,0.0,1.0,2797,0.0,12.0,12.0,12.0,...,0,0,1,0,0,0,0,0,0,0
4066,24,0.0,45.0,0.0,0.0,0,0.0,12.0,12.0,12.0,...,0,0,1,0,0,0,0,0,0,0
4067,25,NaN,NaN,NaN,NaN,1974,0.0,12.0,12.0,12.0,...,0,0,0,0,0,0,0,0,0,1
